In [1]:
#HTML과 XML 문서를 파싱하기위한 파이썬 패키지 다운로드
#!pip install beautifulsoup4 
#!pip install selenium

# selenium 패키지를 사용하기 위한 Chrome Driver 다운로드
1. Chrome 다운로드
2. 버전에 맞는 Chrome Driver 다운로드 
    * Chrome 버전 확인: 크롬 > 더보기 > 도움말 > Chrome 정보
    * Chrome Driver 다운로드: https://sites.google.com/a/chromium.org/chromedriver/downloads
3. Chrome Driver를 현재 소스코드와 동일한 디렉토리로 이동

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote_plus
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time
import os

In [3]:
NUM = 100000 #크롤링할 이미지 최대 개수(이미지는 NUM이하로 크롤링됨)

In [4]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(3)

In [5]:
def loadHtml(path):
    driver.get(path)
    driver.implicitly_wait(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup.find_all('img')

In [6]:
def crawlingImage(n,savepath,failStack):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    img = soup.find_all('img')
    print(len(img), n)
    if len(img)<=n:
        # chrome 창 스크롤을 내려서 이미지가 렌더링 되도록 함
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        # 결과 더보기 버튼이 있으면 클릭
        result = driver.execute_script(
            '''
            if(document.getElementsByClassName("mye4qd")[0].getAttribute("value")==="결과 더보기")
                document.getElementsByClassName("mye4qd")[0].click()
            '''
            )
        failStack += 1
        if failStack > 5:
            return n, -1
        else:
            return n, failStack
    
    for i in img[n:]:
        print(n)
        try:
            imgUrl = i['src']
        except KeyError:
            try:
                imgUrl = i['data-src']
            except KeyError:
                continue
        #url에서 이미지를 읽어서 지정된 경로에 저장
        with urlopen(imgUrl) as f:
            with open(savepath + str(n)+'.jpg','wb') as h: # w - write b - binary
                img = f.read()
                h.write(img)
        #저장된 이미지 출력
        #with Image.open(savepath + str(n)+'.jpg') as image:
        #    plt.imshow(image)
        #    plt.show()
        n += 1
        #driver.execute_script("window.scrollTo(0, {})".format(n*(231//4)))
        
        if n > NUM:
            break
    failStack = 0
    return n, failStack

# 화재 이미지 크롤링

### crawling "fire outbreak" image in google

In [7]:
img = loadHtml('https://www.google.com/search?q=fire+outbreak&tbm=isch&ved=2ahUKEwjVwvu_ntzwAhWWAaYKHbZ7BngQ2-cCegQIABAA&oq=fire+outbreak&gs_lcp=CgNpbWcQAzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzIECAAQEzoFCAAQsQM6AggAOggIABCxAxCDAToECAAQHjoGCAAQHhATULe3AVjMyQFg6MoBaAFwAHgAgAGcAYgBog-SAQQwLjE0mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=32uoYJXXDZaDmAW295nABw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

62 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62 62
112 62
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
212 112
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212 212
212 212
312 212
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### crawling "fire outbreak kitchen" image in google

In [8]:
img = loadHtml('https://www.google.com/search?q=fire+outbreak+kitchen&tbm=isch&ved=2ahUKEwjDjevMntzwAhW0yosBHZE0Cs4Q2-cCegQIABAA&oq=fire+outbreak+kitchen&gs_lcp=CgNpbWcQA1CB7ARYgewEYInuBGgAcAB4AIABiAGIAYgBkgEDMC4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=-muoYMOoDbSVr7wPkemo8Aw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img2',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

64 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64 64
114 64
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
214 114
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214 214
214 214
314 214
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### crawling "fire outbreak apartment" image in google

In [9]:
img = loadHtml('https://www.google.com/search?q=fire+outbreak+apartment&tbm=isch&ved=2ahUKEwiR1KnzntzwAhULBJQKHTyBDzQQ2-cCegQIABAA&oq=fire+outbreak+apartment&gs_lcp=CgNpbWcQA1CTmANYk5gDYImaA2gAcAB4AIABkgGIAZIBkgEDMC4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=SmyoYJGHOYuI0AS8gr6gAw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img3',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

61 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61 61
111 61
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
211 111
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211 211
211 211
311 211
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### crawling "fire outbreak school" image in google

In [10]:
img = loadHtml('https://www.google.com/search?q=fire+outbreak+school&tbm=isch&ved=2ahUKEwiLk_eVn9zwAhUJ95QKHfpkB5AQ2-cCegQIABAA&oq=fire+outbreak+school&gs_lcp=CgNpbWcQAzoECAAQEzoICAAQCBAeEBNQ4w5YsBhg4BloAHAAeACAAZsBiAG8BpIBAzAuNpgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=k2yoYMv9Honu0wT6yZ2ACQ&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img4',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

66 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66 66
116 66
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
216 116
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216 216
216 216
216 216
316 216
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### crawling "fire outbreak factory" image in google

In [11]:
img = loadHtml('https://www.google.com/search?q=fire+outbreak+factory&tbm=isch&ved=2ahUKEwiMuJaYn9zwAhUDDpQKHT0TDHoQ2-cCegQIABAA&oq=fire+outbreak+factory&gs_lcp=CgNpbWcQAzoECAAQEzoICAAQCBAeEBM6BggAEAgQHlCu-wFYvZECYPWTAmgBcAB4AIABoAGIAYoPkgEEMC4xNJgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=mGyoYIyMDYOc0AS9prDQBw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img5',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

64 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64 64
114 64
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
214 114
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214 214
214 214
214 214
314 214
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### crawling "fire outbreak shop" image in google

In [12]:
img = loadHtml('https://www.google.com/search?q=fire+outbreak+shop&tbm=isch&ved=2ahUKEwiZlMben9zwAhWyI6YKHanLCTIQ2-cCegQIABAA&oq=fire+outbreak+shop&gs_lcp=CgNpbWcQA1AAWABgh_8EaABwAHgAgAEAiAEAkgEAmAEAqgELZ3dzLXdpei1pbWc&sclient=img&ei=K22oYNnSMLLHmAWpl6eQAw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img6',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

60 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60 60
110 60
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
210 110
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210 210
210 210
210 210
310 210
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### crawling "fire outbreak street" image in google

In [13]:
img = loadHtml('https://www.google.com/search?q=fire+outbreak+street&tbm=isch&ved=2ahUKEwjkhKCGoNzwAhUIvJQKHTQDBNQQ2-cCegQIABAA&oq=fire+outbreak+street&gs_lcp=CgNpbWcQAzoECAAQEzoICAAQCBAeEBNQz5sBWKiiAWCRpQFoAHAAeACAAcABiAHaBpIBAzAuNpgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=f22oYKTJA4j40gS0hpCgDQ&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img7',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

63 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63 63
113 63
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
213 113
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213 213
213 213
213 213
313 213
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### crawling "fire outbreak city" image in google

In [14]:
img = loadHtml('https://www.google.com/search?q=fire+outbreak+city&tbm=isch&ved=2ahUKEwicke22oNzwAhVcyZQKHd_HD38Q2-cCegQIABAA&oq=fire+outbreak+city&gs_lcp=CgNpbWcQAzoFCAAQsQM6AggAOggIABCxAxCDAToECAAQHjoECAAQEzoGCAAQHhATOggIABAIEB4QEzoICAAQBRAeEBNQ8aACWPbGAmCLyAJoAnAAeAGAAcECiAH7GJIBCDAuMjEuMC4xmAEAoAEBqgELZ3dzLXdpei1pbWewAQDAAQE&sclient=img&ei=5G2oYJyPPNyS0wTfj7_4Bw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img8',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

61 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61 61
111 61
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
211 111
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211 211
211 211
311 211
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### crawling "화재" image in google

In [15]:
img = loadHtml('https://www.google.com/search?q=%ED%99%94%EC%9E%AC&tbm=isch&ved=2ahUKEwjOt9G5ptzwAhVoEqYKHSAeAMIQ2-cCegQIABAA&oq=%ED%99%94%EC%9E%AC&gs_lcp=CgNpbWcQAzICCAAyAggAMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCAA6CAgAELEDEIMBOgUIABCxA1D3rgFY37QBYIW2AWgBcAB4AYABxwGIAYEGkgEDMC41mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=NXSoYI7JGuikmAWgvICQDA&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img9',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

84 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84 84
142 84
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
251 142
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251 251
251 251
251 251
360 251
251
252
253
254
255
256
257
258
259
260
261
262


### crawling "화재 초기" image in google

In [16]:
img = loadHtml('https://www.google.com/search?q=%ED%99%94%EC%9E%AC+%EC%B4%88%EA%B8%B0&tbm=isch&ved=2ahUKEwjp5P_8ptzwAhUVMKYKHURfBucQ2-cCegQIABAA&oq=%ED%99%94%EC%9E%AC+%EC%B4%88%EA%B8%B0&gs_lcp=CgNpbWcQAzICCAAyBAgAEBgyBAgAEBgyBAgAEBgyBggAEAgQHjoICAAQsQMQgwE6BQgAELEDULkuWPo3YOg4aAJwAHgCgAGXAYgBiQqSAQMwLjmYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=wnSoYOn7KpXgmAXEvpm4Dg&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/1/img10',failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

65 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65 65
116 65
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
219 116
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219 219
219 219
219 219
319 219
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


# 연기 이미지 크롤링

### Crawling "fire smoke" image in google

In [17]:
img = loadHtml('https://www.google.com/search?q=fire+smoke&tbm=isch&ved=2ahUKEwjt7Lb-o9zwAhUMAqYKHXJeC9MQ2-cCegQIABAA&oq=fire+smoke&gs_lcp=CgNpbWcQAzIECAAQEzIECAAQEzIICAAQBxAeEBMyCAgAEAcQHhATMggIABAHEB4QEzIICAAQBxAeEBMyCAgAEAcQHhATMggIABAHEB4QEzIICAAQBxAeEBMyCAgAEAcQHhATOgUIABCxAzoCCABQ8DBYsjZguDdoAHAAeACAAY8BiAGtBZIBAzAuNZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=oHGoYO2cGIyEmAXyvK2YDQ&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM :
    n, failStack = crawlingImage(n,'./data/crawled_images/2/img', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

61 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61 61
111 61
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
211 111
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211 211
211 211
211 211
311 211
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### Crawling "smoke outbreak apartment" image in google

In [18]:
img = loadHtml('https://www.google.com/search?q=smoke+outbreak+apartment&tbm=isch&ved=2ahUKEwjuuYOqotzwAhUJg5QKHUVZAvkQ2-cCegQIABAA&oq=smoke+outbreak+apartment&gs_lcp=CgNpbWcQA1DjlgJY45YCYJKaAmgAcAB4AIABiAGIAYgBkgEDMC4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=4m-oYO7AOomG0gTFsonIDw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM :
    n, failStack = crawlingImage(n,'./data/crawled_images/2/img2', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

61 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61 61
111 61
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
211 111
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211 211
211 211
211 211
311 211
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### Crawling "smoke in fire" image in google

In [19]:
img = loadHtml('https://www.google.com/search?q=smoke+in+fire&tbm=isch&ved=2ahUKEwjgktmCpNzwAhWQxosBHeQxCTsQ2-cCegQIABAA&oq=smoke+in+fire&gs_lcp=CgNpbWcQAzIECAAQEzIICAAQCBAeEBMyCAgAEAgQHhATMggIABAIEB4QEzoFCAAQsQM6AggAOggIABCxAxCDAToECAAQHlCY3QFYleoBYOXqAWgAcAB4AIABlAGIAeQMkgEEMC4xMpgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=qXGoYKCaFZCNr7wP5OOk2AM&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM :
    n, failStack = crawlingImage(n,'./data/crawled_images/2/img3', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

63 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63 63
113 63
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
213 113
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213 213
213 213
213 213
313 213
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### Crawling "initial smoke in fire" image in google

In [20]:
img = loadHtml('https://www.google.com/search?q=initial+smoke+in+fire&tbm=isch&ved=2ahUKEwj3oL_ipNzwAhUIU5QKHWLlAUoQ2-cCegQIABAA&oq=initial+smoke+in+fire&gs_lcp=CgNpbWcQA1CbOViCVWDzVWgBcAB4AIABlgGIAekLkgEEMC4xMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=cnKoYPefD4im0QTiyofQBA&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM :
    n, failStack = crawlingImage(n,'./data/crawled_images/2/img4', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

64 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64 64
115 64
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
218 115
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218 218
218 218
218 218
318 218
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### Crawling "화재 연기" image in google

In [21]:
img = loadHtml('https://www.google.com/search?q=%ED%99%94%EC%9E%AC+%EC%97%B0%EA%B8%B0&tbm=isch&ved=2ahUKEwjtt6e0p9zwAhUNBaYKHS0mALEQ2-cCegQIABAA&oq=%ED%99%94%EC%9E%AC+%EC%97%B0%EA%B8%B0&gs_lcp=CgNpbWcQAzICCAAyAggAMgIIADICCAAyBAgAEBgyBAgAEBgyBAgAEBgyBAgAEBgyBAgAEBgyBggAEAUQHjoICAAQsQMQgwE6BQgAELEDUPpqWMNzYOB0aAFwAHgCgAHgAYgB3AySAQYwLjEwLjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=NnWoYO3EKo2KmAWtzICICw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM :
    n, failStack = crawlingImage(n,'./data/crawled_images/2/img5', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

67 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67 67
125 67
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
234 125
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234 234
234 234
234 234
343 234
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


# 일반 이미지 크롤링

### Crawling "kitchen" image in google

In [22]:
img = loadHtml('https://www.google.com/search?q=kitchen&tbm=isch&ved=2ahUKEwiYyqDLpdzwAhXXBaYKHYxGDIUQ2-cCegQIABAA&oq=kitchen&gs_lcp=CgNpbWcQAzIFCAAQsQMyAggAMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCABQvY0BWIOUAWDflAFoAHAAeACAAZIBiAG_BpIBAzAuNpgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=TXOoYNjvOdeLmAWMjbGoCA&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

69 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69 69
127 69
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
235 127
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235 235
235 235
343 235
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### Crawling "apartment" image in google

In [23]:
img = loadHtml('https://www.google.com/search?q=apartment&tbm=isch&ved=2ahUKEwig_ZG8p9zwAhUVMKYKHURfBucQ2-cCegQIABAA&oq=apartment&gs_lcp=CgNpbWcQAzIFCAAQsQMyAggAMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCABQq-YDWKvmA2DN6QNoAHAAeACAAZsBiAGbAZIBAzAuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=R3WoYOC9B5XgmAXEvpm4Dg&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img2', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

68 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68 68
126 68
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
235 126
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235 235
235 235
235 235
344 235
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### Crawling "street" image in google

In [24]:
img = loadHtml('https://www.google.com/search?q=street&tbm=isch&ved=2ahUKEwiPt8Lap9zwAhV3yIsBHVKMDv8Q2-cCegQIABAA&oq=street&gs_lcp=CgNpbWcQAzIFCAAQsQMyBQgAELEDMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCABQs25Ys25glHNoAHAAeACAAawBiAGsAZIBAzAuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=hnWoYI_cMveQr7wP0pi6-A8&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img3', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

69 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69 69
127 69
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
236 127
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236 236
236 236
345 236
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### Crawling "factory" image in google

In [25]:
img = loadHtml('https://www.google.com/search?q=factory&tbm=isch&ved=2ahUKEwiPu6Pip9zwAhWvyIsBHf4JAhgQ2-cCegQIABAA&oq=factory&gs_lcp=CgNpbWcQAzICCAAyAggAMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCABQqnxYqnxg3X5oAHAAeACAAZIBiAGSAZIBAzAuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=l3WoYM-TBq-Rr7wP_pOIwAE&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img4', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

82 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82 82
140 82
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
249 140
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249 249
249 249
249 249
358 249
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### Crawling "school" image in google

In [26]:
img = loadHtml('https://www.google.com/search?q=school&tbm=isch&ved=2ahUKEwiWodzqp9zwAhXZy4sBHRLwB3cQ2-cCegQIABAA&oq=school&gs_lcp=CgNpbWcQAzIFCAAQsQMyBQgAELEDMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCABQyGlYyGlgvmxoAHAAeACAAYgBiAGIAZIBAzAuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=qHWoYJatMdmXr7wPkuCfuAc&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img5', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

50 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50 50
100 50
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
200 100
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200 200
200 200
300 200
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### Crawling "shop" image in google

In [27]:
img = loadHtml('https://www.google.com/search?q=shop&tbm=isch&ved=2ahUKEwjEofvxp9zwAhVL7JQKHTptCzUQ2-cCegQIABAA&oq=shop&gs_lcp=CgNpbWcQAzICCAAyAggAMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCABQnXtYnXtgtH5oAHAAeACAAZUBiAGVAZIBAzAuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=t3WoYITqPMvY0wS62q2oAw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img6', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

68 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68 68
123 68
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
229 123
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229 229
229 229
332 229
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### Crawling "city" image in google

In [28]:
img = loadHtml('https://www.google.com/search?q=city&tbm=isch&ved=2ahUKEwjJjKD6p9zwAhWSI6YKHWNGD8AQ2-cCegQIABAA&oq=city&gs_lcp=CgNpbWcQAzIFCAAQsQMyCAgAELEDEIMBMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCABQl0VYl0Vg9kdoAHAAeACAAZcBiAGXAZIBAzAuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=yXWoYImEF5LHmAXjjL2ADA&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img7', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

68 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68 68
126 68
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
235 126
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235 235
235 235
235 235
344 235
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262


### Crawling "단풍" image in google

In [29]:
img = loadHtml('https://www.google.com/search?q=%EB%8B%A8%ED%92%8D&tbm=isch&ved=2ahUKEwjWmqX_p9zwAhUMUpQKHWu0Dr0Q2-cCegQIABAA&oq=%EB%8B%A8%ED%92%8D&gs_lcp=CgNpbWcQAzICCAAyAggAMgIIADIICAAQsQMQgwEyAggAMgIIADICCAAyAggAMgIIADICCABQ83hY83hg_XtoAHAAeACAAYsBiAGLAZIBAzAuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=03WoYJblOYyk0QTr6LroCw&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img8', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

84 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84 84
142 84
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
251 142
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251 251
251 251
360 251
251
252
253
254
255
256
257
258
259
260
261
262
263
264


### Crawling "노을" image in google

In [30]:
img = loadHtml('https://www.google.com/search?q=%EB%85%B8%EC%9D%84&tbm=isch&ved=2ahUKEwiN6LyHqNzwAhUK5JQKHbQjA8AQ2-cCegQIABAA&oq=%EB%85%B8%EC%9D%84&gs_lcp=CgNpbWcQAzIFCAAQsQMyAggAMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCABQrEZYrEZg60loAHAAeACAAY4BiAGOAZIBAzAuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=5XWoYM3hBorI0wS0x4yADA&bih=754&biw=1536&rlz=1C1OKWM_koKR870KR870&hl=ko')
n = 0
failStack = 0
while n<NUM:
    n, failStack = crawlingImage(n,'./data/crawled_images/0/img9', failStack)
    if failStack==-1:
        print("{}개의 이미지 크롤링 성공".format(n))
        break

84 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84 84
142 84
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
251 142
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251 251
251 251
251 251
360 251
251
252
253
254
255
256
257
258
259
260
261
262


# RGB를 제외한 이미지 삭제

In [31]:
cur_dir = os.getcwd()
data_dir = os.path.join(cur_dir, 'data')
crawled_dir = os.path.join(data_dir, 'crawled_images')
class0_dir = os.path.join(crawled_dir,'0')
class1_dir = os.path.join(crawled_dir,'1')
class2_dir = os.path.join(crawled_dir,'2')

normal_img_files = os.listdir(class0_dir)
fire_img_files = os.listdir(class1_dir)
smoke_img_files = os.listdir(class2_dir)

nn=0
for img in normal_img_files:
    img_path = os.path.join(class0_dir,img)
    image = Image.open(img_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        print(img, image_mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(img_path)
        nn+=1
print("일반 이미지 {}개 삭제".format(nn))

fn = 0
for img in fire_img_files:
    img_path = os.path.join(class1_dir,img)
    image = Image.open(img_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        print(img, image_mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(img_path)
        fn+=1
print("화재 이미지 {}개 삭제".format(fn))

sn = 0
for img in smoke_img_files:
    img_path = os.path.join(class2_dir,img)
    image = Image.open(img_path)
    image_mode = image.mode
    if image_mode != 'RGB':
        print(img, image_mode)
        image = np.asarray(image)
        print(image.shape)
        os.remove(img_path)
        sn+=1
print("연기 이미지 {}개 삭제".format(sn))


img2458.jpg P
(119, 421)
img2486.jpg P
(225, 225)
img2600.jpg P
(179, 282)
img2636.jpg P
(228, 221)
img2641.jpg P
(159, 318)
img2642.jpg P
(162, 311)
img2662.jpg P
(229, 220)
img2694.jpg P
(183, 275)
img2717.jpg P
(194, 259)
img2789.jpg P
(116, 435)
img2893.jpg P
(225, 225)
img3176.jpg P
(239, 210)
img3251.jpg P
(310, 162)
img3256.jpg P
(241, 209)
img3338.jpg P
(175, 289)
img3345.jpg P
(163, 310)
img3370.jpg P
(200, 200)
img3431.jpg P
(177, 285)
img3579.jpg P
(225, 225)
img3653.jpg P
(108, 467)
img3657.jpg P
(101, 500)
img3682.jpg P
(99, 506)
img3727.jpg P
(225, 225)
img3732.jpg P
(226, 223)
img3785.jpg P
(119, 424)
img3817.jpg P
(194, 260)
img3819.jpg P
(249, 202)
img3825.jpg P
(98, 512)
img3857.jpg P
(216, 233)
img3861.jpg P
(225, 225)
img3864.jpg P
(225, 225)
img3885.jpg P
(207, 244)
img3905.jpg P
(175, 289)
img3910.jpg P
(189, 267)
img3912.jpg P
(200, 200)
img3922.jpg P
(175, 289)
img41004.jpg P
(146, 220)
img41006.jpg P
(165, 259)
img4107.jpg P
(100, 100)
img412.jpg P
(100, 100)
i

img655.jpg P
(212, 212)
img6550.jpg P
(400, 88)
img6552.jpg P
(159, 318)
img6563.jpg P
(311, 162)
img6567.jpg P
(127, 397)
img6575.jpg P
(179, 282)
img6578.jpg P
(225, 225)
img658.jpg P
(225, 225)
img6581.jpg P
(86, 584)
img6587.jpg P
(225, 225)
img6588.jpg P
(142, 354)
img659.jpg P
(163, 310)
img6590.jpg P
(225, 225)
img6601.jpg P
(167, 302)
img6603.jpg P
(207, 244)
img6604.jpg P
(320, 157)
img6606.jpg P
(178, 282)
img6612.jpg P
(166, 304)
img6621.jpg P
(109, 463)
img6627.jpg P
(109, 461)
img6631.jpg P
(117, 430)
img6635.jpg P
(225, 225)
img664.jpg P
(225, 225)
img6641.jpg P
(125, 402)
img6648.jpg P
(143, 352)
img6649.jpg P
(225, 225)
img6650.jpg P
(100, 186)
img6651.jpg P
(125, 402)
img6655.jpg P
(204, 247)
img666.jpg P
(100, 100)
img6664.jpg P
(106, 476)
img6665.jpg P
(157, 321)
img6669.jpg P
(181, 278)
img6671.jpg P
(120, 420)
img6672.jpg P
(143, 352)
img6674.jpg P
(200, 200)
img6676.jpg P
(200, 200)
img6678.jpg P
(266, 190)
img6683.jpg P
(223, 226)
img6684.jpg P
(181, 279)
img6688

(224, 224)
img9638.jpg P
(213, 237)
img9639.jpg P
(226, 223)
img9655.jpg P
(308, 164)
img9657.jpg P
(179, 281)
img9658.jpg P
(183, 275)
img9668.jpg P
(150, 337)
img9671.jpg P
(139, 363)
img9674.jpg P
(160, 315)
img9707.jpg P
(270, 187)
img9711.jpg P
(224, 224)
img9713.jpg P
(255, 198)
img9721.jpg P
(153, 257)
img9730.jpg P
(137, 367)
img9738.jpg P
(183, 275)
img9745.jpg P
(225, 225)
img9746.jpg P
(218, 231)
img9748.jpg P
(184, 274)
img9750.jpg P
(172, 293)
img9761.jpg P
(234, 216)
img9766.jpg P
(157, 320)
img9772.jpg P
(146, 344)
img9778.jpg P
(283, 178)
img9788.jpg P
(180, 279)
img9798.jpg P
(267, 189)
img9804.jpg P
(137, 368)
img9809.jpg P
(136, 369)
img9821.jpg P
(225, 225)
img9823.jpg P
(183, 275)
img9830.jpg P
(225, 225)
img9831.jpg P
(272, 185)
img9836.jpg P
(225, 225)
img9840.jpg P
(209, 241)
img9849.jpg P
(125, 402)
img9852.jpg P
(185, 273)
img9857.jpg P
(225, 225)
img9861.jpg P
(205, 245)
img9870.jpg P
(159, 318)
img9872.jpg P
(149, 338)
img9875.jpg P
(255, 197)
img9881.jpg P
